# Building Fast Queries on a CSV

## Introduction

We run an online laptop store and would like to build a way to answer a few different business questions about our inventory.

We will use the `laptops.csv` file as our inventory, which has been adapted from the original [dataset](https://www.kaggle.com/datasets/muhammetvarl/laptop-price).

Our dataset contains the following column titles:

- `Id`: A unique identifier for the laptop
- `Company`: The name of the company that produces the laptop
- `Product`: The name of the laptop
- `TypeName`: The type of laptop
- `Inches`: The size of the screen in inches
- `ScreenResolution`: The resolution of the screen
- `Cpu`: The laptop CPU
- `Ram`: The amount of RAM in the laptop
- `Memory`: The size of the hard drive
- `Gpu`: The graphics card name
- `OpSys`: The name of the operating system
- `Weight`: The laptop weight
- `Price`: The price of the laptop

## Reading the inventory

First, we'll read in our file and separate the header from the remaining rows.

We'll then display the header and the first few rows.

In [100]:
import csv

with open("laptops.csv") as file:
    reader = csv.reader(file)
    data = list(reader)

header = data[0]
rows = data[1:]

print(header, "\n")

for row in rows[:5]:
    print(row, "\n")

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price'] 

['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'] 

['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'] 

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'] 

['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'] 

['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '2

## Inventory class

We are going to create a class that represents our inventory, with its class methods implementing the queries that we want to answer about our inventory.

We'll start by implementing the constructor, which will take the name of the CSV file as an argument and then read the rows contained in it. The header and rows will be read into `self.header` and `self.rows`, followed by converting the `price` of each row *(the last column)* to an integer. 

The class will be tested by creating an instance of it, using `laptops.csv` as the argument.

In [101]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            data = list(reader)
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])

inventory = Inventory("laptops.csv")
print(inventory.header, "\n")
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price'] 

1303


## Finding a laptop from the Id

To implement a way to look up a laptop from a given identifier, we'll create a `get_laptop_from_id` method. 

We'll then test this method with a few example identifiers. 

In [102]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            data = list(reader)
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])

    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None

inventory = Inventory("laptops.csv")
print(inventory.get_laptop_from_id("3362737"), "\n")
print(inventory.get_laptop_from_id("3362736"))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575] 

None


## Improving Id lookups

In the worst cases, the `get_laptop_from_id` method requires us to look at every single row to find the identifier we are looking for, or to conclude that there is no match. 

To improve the process of looking up a given identifier, we'll preprocess the data into a dictionary, where the keys are the Id's and the values are the rows. 

We'll use the dictionary in an alternative method, `get_laptop_from_id_fast`, and then test it with a few example identifiers.

In [103]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            data = list(reader)
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row

    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None

    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

inventory = Inventory("laptops.csv")
print(inventory.get_laptop_from_id_fast("3362737"), "\n")
print(inventory.get_laptop_from_id_fast("3362736"))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575] 

None


## Comparing the performance

Next, we'll experiment to compare the performance of the two methods. 

We'll start by generating random Id's and then use both methods to lookup these same Id's. We'll measure the execution time of each lookup and for each method, add all of the times together.

In [104]:
import time
import random

ids = [str(random.randint(1000000, 9999999)) for num in range(10000)]
inventory = Inventory("laptops.csv")

total_time_no_dict = 0
for identifier in ids:
    start = time.time()
    inventory.get_laptop_from_id(identifier)
    end = time.time()
    total_time_no_dict += (end - start)

total_time_dict = 0
for identifier in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(identifier)
    end = time.time()
    total_time_dict += (end - start)

print(total_time_no_dict)
print(total_time_dict)

0.20449590682983398
0.0013420581817626953
